## Arbol de clasificación



In [1]:
# Importar modulos para el tratamiento de los datos y visualizaciones
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


In [2]:
# Cargar datos
df = pd.read_csv("Data/robot_wall_follower/sensor_readings_4.csv", header=None,
                 names=["sd_front", "sd_left", "sd_right", "sd_back", "target"])


In [3]:
df.head()

,sd_front,sd_left,sd_right,sd_back,target
0,1.687,0.445,2.332,0.429,Slight-Right-Turn
1,1.687,0.449,2.332,0.429,Slight-Right-Turn
2,1.687,0.449,2.334,0.429,Slight-Right-Turn
3,1.687,0.449,2.334,0.429,Slight-Right-Turn
4,1.687,0.449,2.334,0.429,Slight-Right-Turn
